Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
logistic_graph = tf.Graph()
with logistic_graph.as_default():
    
    tf_l2_reg  = tf.constant( 1e-2 )
    tf_train_x = tf.placeholder(tf.float32, shape=[None, image_size*image_size])
    tf_train_y = tf.placeholder(tf.float32, shape=[None, num_labels])
    tf_keep_prob = tf.placeholder(tf.float32)
    
    w1_fc = tf.Variable( tf.truncated_normal([image_size*image_size, num_labels]) )
    b1_fc = tf.Variable( tf.zeros(num_labels) )
    
    logits = tf.nn.dropout(
        (tf.matmul( tf_train_x, w1_fc ) + b1_fc), tf_keep_prob )
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( 
        labels = tf_train_y, logits = logits ) + tf.nn.l2_loss(w1_fc)*tf_l2_reg )
    
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_pred = tf.nn.softmax(logits)

In [6]:
num_steps = 801
train_subset = 10000

with tf.Session(graph = logistic_graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        train_feed_dict = { tf_train_x: train_dataset[:train_subset,:], 
                            tf_train_y: train_labels[:train_subset],
                            tf_keep_prob: 1.0 }
        _, l, pred = sess.run([train_step, loss, train_pred], 
                              feed_dict = train_feed_dict)
        if step % 100 == 0:
            train_acc = accuracy(pred, train_labels[:train_subset])
            print ('Loss at step %d: train_loss %f, train_acc %.1f%%'% (step, l, train_acc))
            valid_feed_dict = { tf_train_x: valid_dataset, 
                                tf_train_y: valid_labels,
                                tf_keep_prob: 1.0}            
            valid_pred = sess.run(train_pred, feed_dict = valid_feed_dict)
            valid_acc = accuracy( valid_pred, valid_labels )
            print ('  valid_acc %.1f%%'% (valid_acc) )
    test_feed_dict = { tf_train_x: test_dataset, 
                       tf_train_y: test_labels,
                       tf_keep_prob: 1.0}
    test_pred = sess.run( train_pred, feed_dict = test_feed_dict )
    test_acc = accuracy( test_pred, test_labels )
    print ('Test accuracy: %.1f%%'% (test_acc))
            
        

Initialized
Loss at step 0: train_loss 48.574844, train_acc 7.9%
  valid_acc 10.7%
Loss at step 100: train_loss 11.776622, train_acc 72.8%
  valid_acc 72.6%
Loss at step 200: train_loss 4.445036, train_acc 78.7%
  valid_acc 77.5%
Loss at step 300: train_loss 1.964018, train_acc 82.2%
  valid_acc 80.4%
Loss at step 400: train_loss 1.123256, train_acc 83.7%
  valid_acc 82.1%
Loss at step 500: train_loss 0.834843, train_acc 84.2%
  valid_acc 82.7%
Loss at step 600: train_loss 0.734666, train_acc 84.4%
  valid_acc 82.9%
Loss at step 700: train_loss 0.699533, train_acc 84.5%
  valid_acc 83.0%
Loss at step 800: train_loss 0.687121, train_acc 84.6%
  valid_acc 83.0%
Test accuracy: 88.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 801
train_subset = 100

with tf.Session(graph = logistic_graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        train_feed_dict = { tf_train_x: train_dataset[:train_subset,:], 
                            tf_train_y: train_labels[:train_subset],
                            tf_keep_prob: 1.0}
        _, l, pred = sess.run([train_step, loss, train_pred], 
                              feed_dict = train_feed_dict)
        if step % 100 == 0:
            train_acc = accuracy(pred, train_labels[:train_subset])
            print ('Loss at step %d: train_loss %f, train_acc %.1f%%'% (step, l, train_acc))
            valid_feed_dict = { tf_train_x: valid_dataset, 
                                tf_train_y: valid_labels,
                                tf_keep_prob: 1.0 }            
            valid_pred = sess.run(train_pred, feed_dict = valid_feed_dict)
            valid_acc = accuracy( valid_pred, valid_labels )
            print ('  valid_acc %.1f%%'% (valid_acc) )
    test_feed_dict = { tf_train_x: test_dataset, 
                       tf_train_y: test_labels,
                       tf_keep_prob: 1.0 }
    test_pred = sess.run( train_pred, feed_dict = test_feed_dict )
    test_acc = accuracy( test_pred, test_labels )
    print ('Test accuracy: %.1f%%'% (test_acc))

Initialized
Loss at step 0: train_loss 42.487843, train_acc 13.0%
  valid_acc 15.6%
Loss at step 100: train_loss 11.011569, train_acc 100.0%
  valid_acc 51.0%
Loss at step 200: train_loss 4.105383, train_acc 100.0%
  valid_acc 56.4%
Loss at step 300: train_loss 1.583164, train_acc 100.0%
  valid_acc 61.2%
Loss at step 400: train_loss 0.664014, train_acc 100.0%
  valid_acc 64.3%
Loss at step 500: train_loss 0.328745, train_acc 100.0%
  valid_acc 66.0%
Loss at step 600: train_loss 0.206145, train_acc 100.0%
  valid_acc 67.0%
Loss at step 700: train_loss 0.161095, train_acc 100.0%
  valid_acc 67.0%
Loss at step 800: train_loss 0.144370, train_acc 100.0%
  valid_acc 67.0%
Test accuracy: 72.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
num_steps = 801
train_subset = 100

with tf.Session(graph = logistic_graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        train_feed_dict = { tf_train_x: train_dataset[:train_subset,:], 
                            tf_train_y: train_labels[:train_subset],
                            tf_keep_prob: 0.25}
        _, l, pred = sess.run([train_step, loss, train_pred], 
                              feed_dict = train_feed_dict)
        if step % 100 == 0:
            train_acc = accuracy(pred, train_labels[:train_subset])
            print ('Loss at step %d: train_loss %f, train_acc %.1f%%'% (step, l, train_acc))
            valid_feed_dict = { tf_train_x: valid_dataset, 
                                tf_train_y: valid_labels,
                                tf_keep_prob: 1.0 }            
            valid_pred = sess.run(train_pred, feed_dict = valid_feed_dict)
            valid_acc = accuracy( valid_pred, valid_labels )
            print ('  valid_acc %.1f%%'% (valid_acc) )
    test_feed_dict = { tf_train_x: test_dataset, 
                       tf_train_y: test_labels,
                       tf_keep_prob: 1.0 }
    test_pred = sess.run( train_pred, feed_dict = test_feed_dict )
    test_acc = accuracy( test_pred, test_labels )
    print ('Test accuracy: %.1f%%'% (test_acc))

Initialized
Loss at step 0: train_loss 61.717518, train_acc 7.0%
  valid_acc 13.3%
Loss at step 100: train_loss 12.451206, train_acc 35.0%
  valid_acc 42.1%
Loss at step 200: train_loss 5.602417, train_acc 33.0%
  valid_acc 45.2%
Loss at step 300: train_loss 2.958920, train_acc 37.0%
  valid_acc 50.8%
Loss at step 400: train_loss 2.119291, train_acc 35.0%
  valid_acc 57.1%
Loss at step 500: train_loss 1.873753, train_acc 28.0%
  valid_acc 61.7%
Loss at step 600: train_loss 1.674589, train_acc 33.0%
  valid_acc 62.8%
Loss at step 700: train_loss 1.754251, train_acc 25.0%
  valid_acc 64.3%
Loss at step 800: train_loss 1.619933, train_acc 34.0%
  valid_acc 64.1%
Test accuracy: 69.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
